This notebook contains the processing pipeline for performing probabilistic CSD on PROCAN data using Dipy.

In [1]:
import numpy as np

from os.path import join
from os import listdir
import fnmatch
import sys
import time

import nibabel as nib

In [2]:
from dipy.segment.mask import median_otsu
from dipy.core.histeq import histeq
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table
from dipy.reconst.csdeconv import recursive_response
import dipy.reconst.dti as dti
from dipy.reconst.csdeconv import ConstrainedSphericalDeconvModel
from dipy.direction import ProbabilisticDirectionGetter
from dipy.data import default_sphere
from dipy.tracking.local import ThresholdTissueClassifier
from dipy.tracking import utils
from dipy.tracking.local import LocalTracking
from dipy.viz.colormap import line_colors
from dipy.io.trackvis import save_trk    
from dipy.io.pickles import save_pickle

/home/ben/anaconda2/lib/python2.7/site-packages/skimage/filter/__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '


In [14]:
root = '/hdd/PROCAN_DTI'
dirs = listdir(root)
# p_list = ['0006','0007','0009','0010','0011','0017',]
p_list = ['0024']
streams = []
for i in range(len(p_list)):
    pdir = 'CAL' + p_list[i] + '_BL'
    try:
        print "\n----" + pdir + '----'
        
        b1000_dir = None
        b2500_dir = None
        
        bval_1000 = None
        bval_2500 = None
        
        bvec_1000 = None
        bvec_2500 = None        
        
        path = join(root, pdir)
        
        for file in listdir(path):
            if fnmatch.fnmatch(file, '*1000*'):
                b1000_dir = file
            elif fnmatch.fnmatch(file, '*2500*'):
                b2500_dir = file
                
        print "B1000:", join(path,b1000_dir)      
        
        b1000_out_dir = None
        for file in listdir( join(path,b1000_dir) ):
            if fnmatch.fnmatch(file, 'DTI_1000_process'):
                b1000_out_dir = join(path, b1000_dir, file)
                break
                
        print "Processed folder:", b1000_out_dir
                
        for file in listdir(b1000_out_dir):
            if fnmatch.fnmatch(file, '*.bval'):
                bval_1000 = file
            elif fnmatch.fnmatch(file, '*.bvec'):
                bvec_1000 = file
        
        fdwi = join(path, b1000_out_dir, 'data.nii')
        fbval = join(path, b1000_out_dir, bval_1000)
        fbvec = join(path, b1000_out_dir, bvec_1000)
        
        print "DWI location: ", fdwi
        print "b-value location: ", fbval
        print "b-vector location: ", fbvec                
          
        streamlines = process_DWI(pdir+'_B1000', fdwi, fbval, fbvec, path)
#         streams.append(streamlines)
        
    except OSError as e:
        print e.strerror
    


----CAL0024_BL----
B1000: /hdd/PROCAN_DTI/CAL0024_BL/DTI_ASSET45dir_B1000_17
Processed folder: /hdd/PROCAN_DTI/CAL0024_BL/DTI_ASSET45dir_B1000_17/DTI_1000_process
DWI location:  /hdd/PROCAN_DTI/CAL0024_BL/DTI_ASSET45dir_B1000_17/DTI_1000_process/data.nii
b-value location:  /hdd/PROCAN_DTI/CAL0024_BL/DTI_ASSET45dir_B1000_17/DTI_1000_process/20150716_141017JAFMRIPROCANv32015MAs017a1001.bval
b-vector location:  /hdd/PROCAN_DTI/CAL0024_BL/DTI_ASSET45dir_B1000_17/DTI_1000_process/20150716_141017JAFMRIPROCANv32015MAs017a1001.bvec
Data shape:  (256, 256, 63, 53)
Voxel size:  (0.85939997, 0.85939997, 2.2)
Gradient table: B-values shape (53,)
         min 0.000000 
         max 1000.000000 
B-vectors shape (53, 3)
         min -1.000000 
         max 1.000000 
 None
Time to fit tensor model: 59.6943500042
FA/MD calc time: 60.0819091797
Response function calc time: 846.004232168
CSD fit time: 379.864629984
Probabilistic direction getter time: 3.83863306046
Tracking time: 0.000173091888428
Saved

In [13]:
def process_DWI(pid, fdwi, fbval, fbvec, path):
    call_time = time.time()
    img = nib.load(fdwi)    
    data_raw = img.get_data()
    print "Data shape: ", data_raw.shape
    print "Voxel size: ", img.header.get_zooms()[:3]
    
    b0_mask, mask = median_otsu(data_raw, 2, 1)

    mask_img = nib.Nifti1Image(mask.astype(np.float32), img.affine)
    b0_img = nib.Nifti1Image(b0_mask.astype(np.float32), img.affine)

    data = b0_mask
    
    # We need the b-values and b-vectors 
    bvals, bvecs = read_bvals_bvecs(fbval, fbvec)

    # Construct the GradientTable which Dipy uses to hold all acquisition 
    # specifc parameters: bvals, bvecs, timings, etc.
    gtab = gradient_table(bvals, bvecs)
    print "Gradient table: ", gtab.info
        
    st=time.time()
    tenmodel = dti.TensorModel(gtab)
    # tenfit = tenmodel.fit(data, mask=data[...,0] > 200)  # Where does this mask come from? what are we masking on? 
    tenfit = tenmodel.fit(data, mask=data[...,0] > 400) 
    print "Time to fit tensor model:", time.time()-st
    
    FA = dti.fractional_anisotropy(tenfit.evals)
    MD = dti.mean_diffusivity(tenfit.evals)
    print "FA/MD calc time:", time.time()-st
    wm_mask = (np.logical_or(FA >= 0.4, (np.logical_and(FA >= 0.15, MD >= 0.0011)))) # Original mask from tutorial
    
    response = recursive_response(gtab, data, mask=wm_mask, sh_order=8, 
                                 peak_thr=0.01, init_fa=0.08,
                                 init_trace=0.0021, iter=8, convergence=0.001,
                                 parallel=True)
    print "Response function calc time:", time.time()-st
    
    st = time.time()
    csd_model = ConstrainedSphericalDeconvModel(gtab, response)

    csd_fit = csd_model.fit(data)
    print "CSD fit time:", time.time()-st
    
    st = time.time()
    prob_dg = ProbabilisticDirectionGetter.from_shcoeff(csd_fit.shm_coeff,
                                                     max_angle=30.,
                                                     sphere=default_sphere)
    print "Probabilistic direction getter time:", time.time()-st
    
    classifier = ThresholdTissueClassifier(FA, 0.2)
    affine = img.affine
    seeds = utils.seeds_from_mask(wm_mask, density=[1,1,1], affine=affine)
    
    # Init LocalTracking
    st = time.time()
    streamlines = LocalTracking(prob_dg, classifier, seeds, affine, step_size=0.5, max_cross=1)
    print "Tracking time:", time.time()-st
    # print list(streamlines)

    # store streamlines as a list:
    streamlines = list(streamlines)
    color = line_colors(streamlines)
    
    # Save the streamlines to a trackvis file for external processing
    trk_path = join(path, pid + ".trk")
    save_trk(trk_path, streamlines, affine, data[...,0].shape)
    print "Saved .trk to", trk_path
    print "Total process time:", time.time()-call_time
    
    # Pickle the streamlines
#     pick_path = join(path, pid + ".p")
    
#     st = time.time()
#     save_pickle(pick_path, [streamlines])
#     print "Pickle time:", time.time()-st

    return streamlines

In [19]:
# from dipy.data import get_data
# fname = get_data('/hdd/PROCAN_DTI/CAL0005_BL/CAL0005_BL_B1000.trk')
# print fname
st = time.time()
strs, hdr = nib.trackvis.read('/hdd/PROCAN_DTI/CAL0005_BL/CAL0005_BL_B1000.trk')
print "Time to read .trk:", time.time()-st
str_list = [x[0] for x in strs]

Time to read .trk: 1.63856601715
182613
(array([[  45.26596451,  108.07085419,   71.07394409],
       [  45.11849976,  107.85469818,   71.5       ],
       [  45.03215408,  107.60952759,   71.92713165]], dtype=float32), None, None)


In [25]:
ren = fvtk.ren()
streamlines_actor = fvtk.line(str_list, line_colors(str_list))
fvtk.add(ren, streamlines_actor)
fvtk.show(ren)